<a href="https://colab.research.google.com/github/sonmh79/VRPPD-with-robots/blob/main/Delivery_Robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import time
import random

In [22]:
e1 = pd.read_excel('/content/drive/MyDrive/small_sized_210718.xlsx',sheet_name='seed1',header=None)


In [25]:
e1.loc[9][0]

'dd =  {1: 2, 2: 2, 3: 1, 4: 1, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2}'

In [34]:
dd =  {1: 2, 2: 2, 3: 1, 4: 1, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2}
df = pd.DataFrame(dd,index=["Delivery demand"]).T
df

,Delivery demand
1,2
2,2
3,1
4,1
5,2
6,2
7,2
8,2
9,2


In [46]:
d_data = []
data = []
for d in list(e1.loc[15:104][0]):
  data.append(float(d.split(":")[1].rstrip(",").rstrip("}")))
  if len(data) == 9:
    d_data.append(data)
    data = []

for i in range(len(d_data)):
  d_data[i].insert(i,0)
  
d_data = pd.DataFrame(d_data)
d_data

,0,1,2,3,4,5,6,7,8,9
0,0.00,61.55,97.69,162.55,109.69,150.69,144.69,213.83,155.55,218.83
1,61.55,0.00,100.69,100.69,152.55,137.83,122.55,162.69,209.69,167.55
2,97.69,100.69,0.00,160.69,120.55,190.69,182.69,197.55,149.83,195.83
3,162.55,100.69,160.69,0.00,94.55,116.69,110.55,146.69,146.55,164.69
4,109.69,152.55,120.55,94.55,0.00,93.69,98.83,145.55,125.69,192.55
5,150.69,137.83,190.69,116.69,93.69,0.00,98.83,109.55,152.83,178.83
6,144.69,122.55,182.69,110.55,98.83,98.83,0.00,126.83,107.69,148.55
7,213.83,162.69,197.55,146.69,145.55,109.55,126.83,0.00,52.69,107.69
8,155.55,209.69,149.83,146.55,125.69,152.83,107.69,52.69,0.00,160.55
9,218.83,167.55,195.83,164.69,192.55,178.83,148.55,107.69,160.55,0.00


In [133]:
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


class VRP:
  def __init__(self,dataset,d_data = d_data,capacity=3,cnum=100,mutation_prob=0.2,ev_times=100):
    self.n = len(d_data)
    self.c = capacity
    self.data = dataset
    self.d_data = d_data
    self.cnum = cnum
    self.cnt = 0
    self.mutation_prob = mutation_prob
    self.ev_times = ev_times
    self.dist = 0
    self.min_route = []
    self.info = []

  def _chromo(self,t=1):

    """ 1. Make Random Chromos """

    res = []
    for _ in range(t):
     res.append(random.sample(range(1,self.n),self.n-1)) 
    return res

  
  def calc_trips(self,trips):
    dist_list = []
    for trip in trips:
      d = 0
      for i in range(len(trip)-1):
        d += self.d_data.loc[trip[i]][trip[i+1]]
      dist_list.append(d)
    
    res = list(zip(dist_list,trips))
    res.sort()
    return res
  
  def split_trip(self, trip):

    """ Split Trip to Trips """

    stack = 0
    trips = []
    t = [0]
    for node in trip:
      if node == 0:
        stack +=1
      else:
        t.append(node)
      if stack == 2:
        stack = 1
        t.append(0)
        trips.append(t)
        t = [0]
    return trips
  
  def show_trip(self,route):

    """ Print the Trip of Min Route """

    stack = 0
    new_trips = [0]
    for i in range(len(route)):
      cur_demand = self.data.loc[route[i]]["Delivery demand"]
      if stack + cur_demand <= self.c:
        stack += cur_demand
        new_trips.append(route[i])
      else:
        stack= cur_demand
        new_trips = new_trips+[0]
        new_trips.append(route[i])
    new_trips += [0]
    return new_trips


  def _calc_route(self,route):

    """ 2-1-1 Calculate Distance Considering D Demands """
    stack = 0
    new_trips = [0]
    for i in range(len(route)):
      cur_demand = self.data.loc[route[i]]["Delivery demand"]
      if stack + cur_demand <= self.c:
        stack += cur_demand
        new_trips.append(route[i])
      else:
        stack= cur_demand
        new_trips = new_trips+[0]
        new_trips.append(route[i])
    new_trips += [0]

    res = 0
    for i in range(len(new_trips)-1):
      res += self.d_data.loc[new_trips[i]][new_trips[i+1]]
    return res

  def _getdist(self, chromos):

    """ 2-1 Return Distance of Each Chromo """

    dist_list = []

    for chromo in chromos:
      dist_list.append(self._calc_route(chromo))

    fit = list(zip(dist_list,chromos))
    fit.sort(key = lambda x:x[0])
    return fit

  def _rand(self,x,y):
    return int(random.uniform(x,y))

  def _crossover(self,p1,p2,hard_mode=False):

    """ 2-2-2. Crossover and Mutation """

    swap_point = self._rand(1,len(p1))
    c1,c2 = [],[]

    i = 0
    while i < swap_point:
      c1.append(p1[i])
      c2.append(p2[i])
      i+=1

    for e in p2:
      if e not in c1:
        c1.append(e)

    for e in p1:
      if e not in c2:
        c2.append(e)
    
    #mutation
    if random.uniform(0,1) <= self.mutation_prob and len(c1)>2 and not hard_mode:
      e1,e2 = random.sample(range(1,len(c1)),2)
      #target will select mutation child
      if e1 > e2:
        target = e1
      else:
        target = e2
      
      if target%2 == 0:
        c1[e1],c1[e2] = c1[e2],c1[e1]
      else:  
        c2[e1],c2[e2] = c2[e2],c2[e1]

    if hard_mode:
      e1,e2 = random.sample(range(0,len(c1)-1),2)
      c1[e1],c1[e1+1] = c1[e1+1],c1[e1]
      c2[e2],c2[e2+1] = c2[e2+1],c2[e2]

    next_gen = self._getdist([c1,c2,p1,p2])
    #print("next_gen: ",next_gen)
    c1,c2 = next_gen[0][1],next_gen[1][1]
    #print("c1,c2: ",c1,c2)
    return [c1,c2]

  
  def _select_parent(self,parents):

    """ 2-2-1. Select Parents with Roulette Algorithm  """

    fitness = [1/route[0] for route in parents] 
    sum = 0

    for f in fitness:
      sum += f
    p_index = set()

    while len(p_index)<2:
      fit = 0
      target = random.uniform(0,sum)
      for i in range(len(fitness)):
        fit += fitness[i]
        if fit > target:
          p_index.add(i)
          break  
    return list(p_index)

  
  def _make_child(self,dist_routes):

    """ 2-2. Compare Parents with Childs and Return Best Routes  """

    self.cnt += 1
    parents = dist_routes.copy()
    routes = [route[1] for route in parents]
    child = []

    # select top 100 chromos
    while parents and len(child)<self.cnum:
      selected_parents = self._select_parent(parents)
      p1,p2 = routes.pop(selected_parents[0]),routes.pop(selected_parents[1]-1)
      parents.pop(selected_parents[0]),parents.pop(selected_parents[1]-1)
      c1,c2 = self._crossover(p1,p2)
      child += c1,c2

    return child

  def evolution(self,chromo):

    """ 2. Make Childs """
    
    dist_routes = self._getdist(chromo) #(dist,routes)
    child = self._make_child(dist_routes)
    return child

  def ga(self): 

    """ 0. Start Genetic Algorithm !! """

    chromos = self._chromo(100)    
    gen = self.evolution(chromos)
    self.info = self._getdist(gen)
    print("first genenration minimun distance: ",(self.info[0][0],self.info[0][1]))
    for i in range(self.ev_times):
      gen = self.evolution(gen)
      self.info = self._getdist(gen)
      t = self.cnt
      if t % 10 == 0:
        print("{}/{}th evolution min distance: ".format(t,self.ev_times),(self.info[0][0],self.info[0][1]))
      
      # to avoid local minimum
      if t == 30:
        self.mutation_prob = t/100
        print("==========Mutation prob upgrade!==========: ",self.mutation_prob)
      elif t == 50:
        self.mutation_prob = t/100
        print("==========Mutation prob upgrade!==========: ",self.mutation_prob)
      elif t == 70:
        self.mutation_prob = t/100
        print("==========Mutation prob upgrade!==========: ",self.mutation_prob)
      
    self.dist,self.min_route = self.info[0][0],self.info[0][1]   
    print("----------------------------------------Finish----------------------------------------")
    print("Total travel time / Min route: ",self.dist," / ",self.min_route)
    print(self.show_trip(self.info[0][1]))
#if __name__ == "__main__":
  


In [140]:
# small data seed1
v = VRP(df,d_data,capacity=3,ev_times=10)
s= time.time()
v.ga()
trips = v.split_trip(v.show_trip(v.info[0][1]))
trips_info = v.calc_trips(trips)
robot = [[] for _ in range(2)]
score = [0 for _ in range(2)] 
for info in trips_info:
  index = score.index(min(score))
  score[index] += info[0]
  robot[index].append(info[1])

print("robot1: ",robot[0], "Time: ", score[0])
print("robot2: ",robot[1], "Time: ", score[1])
e = time.time()
print("Total time: ",e-s,"(s)")

first genenration minimun distance:  (2233.76, [3, 7, 5, 4, 1, 6, 2, 9, 8])
10/10th evolution min distance:  (2233.7599999999998, [3, 7, 5, 4, 1, 8, 9, 6, 2])
----------------------------------------Finish----------------------------------------
Total travel time / Min route:  2233.7599999999998  /  [7, 3, 4, 5, 8, 9, 6, 1, 2]
[0, 7, 3, 0, 4, 5, 0, 8, 0, 9, 0, 6, 0, 1, 0, 2, 0]
robot1:  [[0, 1, 0], [0, 6, 0], [0, 4, 5, 0], [0, 7, 3, 0]] Time:  1289.62
robot2:  [[0, 2, 0], [0, 8, 0], [0, 9, 0]] Time:  944.1400000000001
Total time:  10.896484375 (s)
